<a href="https://colab.research.google.com/github/franklinokech/CMP73010-Ass1-2018/blob/master/MER_DATA_WRANGLING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wrangling of Marking and Evaluation Raw Data
## Problem Statement
The current raw data set from the survey cto had issues with how the data was migrated from the survey cto into Gsheets and resulted in some data being poorly formated and some missing values. The BI discovered this issues particularly with the MER_2018 and suggested a thorough QC be done onto the datasets using the original data from the Survey CTO server. This notebook gives the literature of how the wrangling was done to ensure the datasets are cleaned and that no missing values are present. This is key in knowing what operations have been done to a particular farm and what was the output.

## Proposed solution
The proposed solution is to revist the MER datasets is their raw format and do some wrangling to get the key status of the farm after each operation. Key items required is the kcode, number of trees counted and the dbh data. The steps are outlines below
### Steps
1. Import the datasets
2. Select Key Attributes
3. Dealing with missing values
4. Merging the compiled datasets



# Importing Datasets

In [0]:
# Import key data analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Import Libraries for connecting to Google drive
!pip install --upgrade -q gspread

## Get the Evaluation Report v3_WIDE


In [0]:
# Authenticate to Google drive and get the required dataset
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())



In [318]:
# import Evaluation Report v3_WIDE
sh = gc.open('Evaluation Report v3_WIDE')
# Select Spreadsheet
# By title
worksheet_v3 = sh.worksheet('Evaluation Report v3_WIDE')
data_v3 = worksheet_v3.get_all_values()
headers = data_v3.pop(0)

df_v3 = pd.DataFrame(data_v3, columns=headers)

# Preview dataframe
df_v3.head()

,SubmissionDate,starttime,endtime,devicephonenum,username,duration,caseid,uniqueID,treesituation,problem_situation,years_away,farmconditionsame,uniqueidfarmconditionsame,roadgoodorbad,Roadwidth,bridgehills,bridgehills_bridge,bridgehills_hills,bridgehills_narrow,bridgehills_sand,bridgehills_crops,bridgehills_no_obstacles,bridgehills_other,other_obstacles_main_road,soiltype,terrainyesno,bigsmallhills,travelupordown,obstaclestowoodlot,obstaclestowoodlot_bush_roots_stumps,obstaclestowoodlot_crops,obstaclestowoodlot_river,obstaclestowoodlot_sand,obstaclestowoodlot_none,obstaclestowoodlot_other,other_obstacles,bushfactor,homestead,thingsinwoodlot,thingsinwoodlot_crops_trees_property_of_owner,...,dead_ornot_8,tree_number_9,h_9,dead_ornot_9,tree_number_10,h_10,dead_ornot_10,tree_number_11,h_11,dead_ornot_11,tree_number_12,h_12,dead_ornot_12,tree_number_13,h_13,dead_ornot_13,tree_number_14,h_14,dead_ornot_14,tree_number_15,h_15,dead_ornot_15,tree_number_16,h_16,dead_ornot_16,tree_number_17,h_17,dead_ornot_17,tree_number_18,h_18,dead_ornot_18,tree_number_19,h_19,dead_ornot_19,tree_number_20,h_20,dead_ornot_20,instanceID,formdef_version,KEY
0,"Feb 1, 2018 10:16:29 AM","Feb 1, 2018 9:12:11 AM","Feb 1, 2018 10:10:42 AM",,samson.ngoba@komaza.org,3293,,erbr62,normal,,,no,,good_road,road_is_wide,hills,0,1,0,0,0,0,0,,loam,terrain_is_flat,,,river,0,0,1,0,0,0,,some_bushes,no,none,0,...,yes,24,12,yes,37,11,yes,39,9,no,40,10,no,42,10,no,21,11,no,22,11,no,23,12,no,10,11,no,11,11,no,12,12,no,4,11,no,uuid:e7955a2a-72c6-40c9-813b-e364761172c6,1801311949,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,"Feb 1, 2018 5:27:13 PM","Feb 1, 2018 10:33:17 AM","Feb 1, 2018 11:10:55 AM",,eunice.gambo@komaza.org,2258,,erbr64,normal,,,no,,good_road,road_is_wide,no_obstacles,0,0,0,0,0,1,0,,sand,terrain_is_flat,,,bush_roots_stumps,1,0,0,0,0,0,,bushes_headhigh,no,none,0,...,no,17,8.7,no,20,12,yes,22,12.7,no,21,10.1,no,23,13.2,no,24,10.7,no,,,,,,,,,,,,,,,,,,,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f,1802010956,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,"Feb 1, 2018 5:27:14 PM","Feb 1, 2018 11:28:07 AM","Feb 1, 2018 5:25:09 PM",,eunice.gambo@komaza.org,1962,,erbr48,clearfell,,,no,,good_road,road_is_wide,hills,0,1,0,0,0,0,0,,sand,terrain_not_flat,small,TravelDown,none,0,0,0,0,1,0,,bushes_headhigh,no,none,0,...,no,,17.4,no,,13,no,,13.6,no,,15.4,no,,12.6,no,,12.7,yes,,12.4,no,,12.7,no,,12.7,no,,17.6,no,,12.4,no,,13.3,yes,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c,1802010956,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,"Feb 1, 2018 5:27:15 PM","Feb 1, 2018 12:23:44 PM","Feb 1, 2018 5:24:52 PM",,eunice.gambo@komaza.org,2486,,erbr37,normal,,,no,,good_road,road_is_wide,no_obstacles,0,0,0,0,0,1,0,,sand,terrain_is_flat,,,crops,0,1,0,0,0,0,,some_bushes,no,crops_trees_property_of_owner,1,...,no,17,12.4,no,16,11.6,no,10,10.6,yes,11,13.6,no,14,10.4,no,15,9.6,no,13,16.3,no,12,16.1,no,5,9,no,1,12.7,no,,,,,,,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf,1802010956,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,"Feb 2, 2018 7:00:07 AM","Feb 1, 2018 10:18:50 AM","Feb 1, 2018 11:44:44 AM",,samson.ngoba@komaza.org,3700,,erbr47,normal,,,no,,good_road,road_is_wide,hills,0,1,0,0,0,0,0,,loam,terrain_not_flat,small,TravelDown,bush_roots_stumps,1,0,0,0,0,0,,no_bushes,no,none,0,...,no,14,7.8,yes,10,7.2,no,11,10.1,no,12,8.8,yes,1,8.9,no,4,9.5,yes,5,7.8,yes,6,7.4,yes,7,7.9,yes,18,8.6,no,,,,,,,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa,1802010956,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


In [319]:
# Get a subset of the dataset
df_v3_filter = df_v3[['SubmissionDate', 'username', 'uniqueID', 'tree_count', 'KEY']]

# Preview the filter
df_v3_filter.head()

,SubmissionDate,username,uniqueID,tree_count,KEY
0,"Feb 1, 2018 10:16:29 AM",samson.ngoba@komaza.org,erbr62,,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,"Feb 1, 2018 5:27:13 PM",eunice.gambo@komaza.org,erbr64,,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,"Feb 1, 2018 5:27:14 PM",eunice.gambo@komaza.org,erbr48,,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,"Feb 1, 2018 5:27:15 PM",eunice.gambo@komaza.org,erbr37,,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,"Feb 2, 2018 7:00:07 AM",samson.ngoba@komaza.org,erbr47,,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


In [320]:
# Rename the columns
df_v3_filter.rename(columns={'SubmissionDate': 'date', 'uniqueID': 'farmid', 'KEY': 'primary_key'}, inplace=True)

# Preview the dataframe
df_v3_filter.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,username,farmid,tree_count,primary_key
0,"Feb 1, 2018 10:16:29 AM",samson.ngoba@komaza.org,erbr62,,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,"Feb 1, 2018 5:27:13 PM",eunice.gambo@komaza.org,erbr64,,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,"Feb 1, 2018 5:27:14 PM",eunice.gambo@komaza.org,erbr48,,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,"Feb 1, 2018 5:27:15 PM",eunice.gambo@komaza.org,erbr37,,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,"Feb 2, 2018 7:00:07 AM",samson.ngoba@komaza.org,erbr47,,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


In [321]:
# Check the data types of the dataset
df_v3_filter.dtypes

date           object
username       object
farmid         object
tree_count     object
primary_key    object
dtype: object

In [322]:
# cast the data types to correct formats

df_v3_filter.date = pd.to_datetime(df_v3_filter.date)
df_v3_filter.tree_count = pd.to_numeric(df_v3_filter.tree_count)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [323]:
# Check the new data types
df_v3_filter.dtypes

date           datetime64[ns]
username               object
farmid                 object
tree_count            float64
primary_key            object
dtype: object

In [324]:
df_v3_filter.head()

,date,username,farmid,tree_count,primary_key
0,2018-02-01 10:16:29,samson.ngoba@komaza.org,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,2018-02-01 17:27:13,eunice.gambo@komaza.org,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,2018-02-01 17:27:14,eunice.gambo@komaza.org,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,2018-02-01 17:27:15,eunice.gambo@komaza.org,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,2018-02-02 07:00:07,samson.ngoba@komaza.org,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


## Get the Evaluation Report v4_WIDE

In [325]:
# import Evaluation Report v3_WIDE
sh = gc.open('Evaluation Report v4_WIDE')
# Select Spreadsheet
# By title
worksheet_v4 = sh.worksheet('Evaluation Report v4_WIDE')
data_v4 = worksheet_v4.get_all_values()
headers = data_v4.pop(0)

df_v4 = pd.DataFrame(data_v4, columns=headers)

# Preview dataframe
df_v4.head()

,SubmissionDate,starttime,endtime,username,duration,team,uniqueID,id,farmername,sublocation,species,farmerphone,nickname,caretakername,plantingyear,count,confirm,mismatch,operation,years_away,similarfarms,uniqueidsimilarfarms,roadgoodorbad,Roadwidth,bridgehills,bridgehills_bridge,bridgehills_hills,bridgehills_narrow,bridgehills_sand,bridgehills_crops,bridgehills_no_obstacles,bridgehills_other,other_obstacles_main_road,soiltype,terrainyesno,bigsmallhills,travelupordown,obstaclestowoodlot,obstaclestowoodlot_bush_roots_stumps,obstaclestowoodlot_crops,...,tree_number_4,height_4,tree_number_5,height_5,tree_number_6,height_6,tree_number_7,height_7,tree_number_8,height_8,tree_number_9,height_9,tree_number_10,height_10,tree_number_11,height_11,tree_number_12,height_12,tree_number_13,height_13,tree_number_14,height_14,tree_number_15,height_15,tree_number_16,height_16,tree_number_17,height_17,tree_number_18,height_18,tree_number_19,height_19,tree_number_20,height_20,instanceID,formdef_version,KEY,review_quality,review_comments,review_corrections
0,"Oct 8, 2018 12:03:54 PM","Oct 8, 2018 9:11:37 AM","Oct 8, 2018 9:43:43 AM",johnson.ngumbao@komaza.org,1791,team_b,crfa553,crfa553,Alphonce shida jefwa,Ndugumnani,Eucalyptus,789767649,Kadena,Farmer,2013,164,not_abletoconfirm,,normal,,no,,bad_road,road_is_wide,hills other,0,1,0,0,0,0,1,"Gullies, rocks",loam,terrain_is_flat,,,bush_roots_stumps,1,0,...,22,11.7,20,12,23,13.2,24,11.5,19,12.1,18,11.1,17,10.7,16,12.6,12,12.3,10,12.4,11,11.9,15,11.6,13,11.8,14,9.5,9,10.6,8,11.7,4,12.5,uuid:c5708dba-f2f5-4ed9-8372-ae838941399e,1809281645,uuid:c5708dba-f2f5-4ed9-8372-ae838941399e,GOOD,"shadrack.oriaro@komaza.org (Oct 8, 2018 3:22:1...",
1,"Oct 8, 2018 12:03:55 PM","Oct 8, 2018 10:04:57 AM","Oct 8, 2018 11:46:59 AM",johnson.ngumbao@komaza.org,2872,team_b,crfa521,crfa521,Kazungu katana buru,Ndugumnani,Eucalyptus,737360624,Babe jumaa,Tatu mudzomba,2014,198,correct,,normal,,no,,bad_road,road_not_wide,narrow sand other,0,0,1,1,0,0,1,Gullies,sand,terrain_is_flat,,,none,0,0,...,40,16.1,41,8.1,8,17,9,12.4,38,15.5,10,10.6,37,13.8,11,15.6,36,11.9,34,10.4,33,14.3,32,12.9,31,15,29,13.6,30,14.6,26,12.4,13,13.5,uuid:38206e0a-e900-4023-8b86-e97683a257d8,1809281645,uuid:38206e0a-e900-4023-8b86-e97683a257d8,GOOD,"shadrack.oriaro@komaza.org (Oct 8, 2018 3:21:5...",
2,"Oct 8, 2018 4:39:56 PM","Oct 8, 2018 11:52:59 AM","Oct 8, 2018 1:20:45 PM",johnson.ngumbao@komaza.org,1253,team_b,crfa523,crfa523,Margaret stephen mwahenge,Ndugumnani,Eucalyptus,0,Margaret,Farmer,2011,37,correct,,clearfell,,no,,bad_road,road_not_wide,narrow sand other,0,0,1,1,0,0,1,Gullies,sand,terrain_is_flat,,,bush_roots_stumps,1,0,...,4,16.7,5,15.8,6,15.9,7,15.7,8,16.2,9,16,10,16.4,11,15.1,12,17.2,13,15.5,14,15.4,15,15.5,16,17.1,17,17.3,18,17.6,19,15.7,20,10.2,uuid:f5dcbb1f-5ab3-4e6f-a83c-d9fedec0af5e,1809281645,uuid:f5dcbb1f-5ab3-4e6f-a83c-d9fedec0af5e,,,
3,"Oct 8, 2018 4:39:57 PM","Oct 8, 2018 12:22:34 PM","Oct 8, 2018 12:24:27 PM",johnson.ngumbao@komaza.org,113,team_b,crfa524,crfa524,Benson matata,Ndugumnani,Eucalyptus,Not Available,Matata,Farmer,2012,80,correct,,tooyoung,more_than_1,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:0d875941-a60c-45e6-8f2b-d9ea1fcd06bc,1809281645,uuid:0d875941-a60c-45e6-8f2b-d9ea1fcd06bc,,,
4,"Oct 8, 2018 4:39:58 PM","Oct 8, 2018 12:31:24 PM","Oct 8, 2018 3:07:18 PM",johnson.ngumbao@komaza.org,1737,team_b,crfa525,crfa525,Kadenge charo mudhengi,Ndugumnani,Eucalyptus,704752513,Kadenge,Farmer,2012,145,not_abletoconfirm,,normal,,no,,bad_road,road_not_wide,narrow sand other,0,0,1,1,0,0,1,Gullies,sand,terrain_is_flat,,,bush_roots_stumps,1,0,...,2,15.9,4,15.3,6,17.1,5,12.8,7,18,11,13.1,10,12.8,19,17.2,24,18.5,25,14.4,26,14.8,27,11.5,18,20,17,12.5,16,15.5,12,16.8,13,15.3,uuid:ca859d39-ac8c-4d85-9bb4-626ba735276f,1809281645,uuid:ca859d39-ac8c-4d85-9bb4-626ba735276f,,,


In [326]:
# Get a subset of the dataset
df_v4_filter = df_v4[['SubmissionDate', 'username', 'uniqueID', 'tree_count', 'KEY']]

# Preview the filter
df_v4_filter.head()

,SubmissionDate,username,uniqueID,tree_count,KEY
0,"Oct 8, 2018 12:03:54 PM",johnson.ngumbao@komaza.org,crfa553,165,uuid:c5708dba-f2f5-4ed9-8372-ae838941399e
1,"Oct 8, 2018 12:03:55 PM",johnson.ngumbao@komaza.org,crfa521,197,uuid:38206e0a-e900-4023-8b86-e97683a257d8
2,"Oct 8, 2018 4:39:56 PM",johnson.ngumbao@komaza.org,crfa523,33,uuid:f5dcbb1f-5ab3-4e6f-a83c-d9fedec0af5e
3,"Oct 8, 2018 4:39:57 PM",johnson.ngumbao@komaza.org,crfa524,,uuid:0d875941-a60c-45e6-8f2b-d9ea1fcd06bc
4,"Oct 8, 2018 4:39:58 PM",johnson.ngumbao@komaza.org,crfa525,146,uuid:ca859d39-ac8c-4d85-9bb4-626ba735276f


In [327]:
# Rename the columns
df_v4_filter.rename(columns={'SubmissionDate': 'date', 'uniqueID': 'farmid', 'KEY': 'primary_key'}, inplace=True)

# Preview the dataframe
df_v4_filter.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,username,farmid,tree_count,primary_key
0,"Oct 8, 2018 12:03:54 PM",johnson.ngumbao@komaza.org,crfa553,165,uuid:c5708dba-f2f5-4ed9-8372-ae838941399e
1,"Oct 8, 2018 12:03:55 PM",johnson.ngumbao@komaza.org,crfa521,197,uuid:38206e0a-e900-4023-8b86-e97683a257d8
2,"Oct 8, 2018 4:39:56 PM",johnson.ngumbao@komaza.org,crfa523,33,uuid:f5dcbb1f-5ab3-4e6f-a83c-d9fedec0af5e
3,"Oct 8, 2018 4:39:57 PM",johnson.ngumbao@komaza.org,crfa524,,uuid:0d875941-a60c-45e6-8f2b-d9ea1fcd06bc
4,"Oct 8, 2018 4:39:58 PM",johnson.ngumbao@komaza.org,crfa525,146,uuid:ca859d39-ac8c-4d85-9bb4-626ba735276f


In [328]:
# Check the data types of the dataset
df_v4_filter.dtypes

date           object
username       object
farmid         object
tree_count     object
primary_key    object
dtype: object

In [329]:
# cast the data types to correct formats

df_v4_filter.date = pd.to_datetime(df_v4_filter.date)
df_v4_filter.tree_count = pd.to_numeric(df_v4_filter.tree_count)

# Preview dtypes
df_v4_filter.dtypes

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


date           datetime64[ns]
username               object
farmid                 object
tree_count            float64
primary_key            object
dtype: object

In [330]:
# Preview the dataframe
df_v4_filter.tail()

,date,username,farmid,tree_count,primary_key
272,2018-12-06 17:06:29,eunice.gambo@komaza.org,stse190,9.0,uuid:7493f3c0-aec7-453e-8966-179d332030b6
273,2018-12-06 17:06:30,eunice.gambo@komaza.org,stse186,1.0,uuid:164f78a4-c3e8-4c6f-8adf-787d7490f26e
274,2018-12-06 17:06:32,eunice.gambo@komaza.org,stse187,39.0,uuid:ca90bef6-ff6d-4cae-92b0-1d056ada5524
275,2018-12-06 17:06:33,eunice.gambo@komaza.org,stse188,24.0,uuid:663000e7-9c9c-4ba7-9083-9df372ddf676
276,2018-12-06 17:06:33,eunice.gambo@komaza.org,stse189,NaN,uuid:1ffa57f4-c403-4cf4-bd54-f1563b03fa05


## Evaluation Report_WIDE
This represent the 2017 mer. For purpose of this analysis, will call dataframe v2

In [331]:
# import Evaluation Report v3_WIDE
sh = gc.open('Evaluation Report_WIDE')
# Select Spreadsheet
# By title
worksheet_v2 = sh.worksheet('Evaluation Report_WIDE')
data_v2 = worksheet_v2.get_all_values()
headers = data_v2.pop(0)

df_v2 = pd.DataFrame(data_v2, columns=headers)

# Preview dataframe
df_v2.head()

,SubmissionDate,starttime,endtime,devicephonenum,username,duration,caseid,uniqueID,treesituation,farmconditionsame,uniqueidfarmconditionsame,roadgoodorbad,Roadwidth,bridgehills,bridgehills_bridge,bridgehills_hills,bridgehills_narrow,bridgehills_sand,bridgehills_crops,bridgehills_no_obstacles,bridgehills_other,bridgehills__Br_,other_obstacles_main_road,soiltype,terrainyesno,bigsmallhills,travelupordown,obstaclestowoodlot,obstaclestowoodlot_bush_roots_stumps,obstaclestowoodlot_crops,obstaclestowoodlot_river,obstaclestowoodlot_sand,obstaclestowoodlot_none,obstaclestowoodlot_other,obstaclestowoodlot_bush,obstaclestowoodlot_C,other_obstacles,bushfactor,homestead,thingsinwoodlot,...,dbh_264,dbh_265,dbh_266,dbh_267,dbh_268,dbh_269,dbh_270,dbh_271,dbh_272,dbh_273,dbh_274,dbh_275,dbh_276,dbh_277,dbh_278,dbh_279,dbh_280,dbh_281,dbh_282,dbh_283,dbh_284,dbh_285,dbh_286,dbh_287,dbh_288,dbh_289,dbh_290,dbh_291,dbh_292,dbh_293,dbh_294,dbh_295,dbh_296,dbh_297,dbh_298,dbh_299,dbh_300,instanceID,formdef_version,KEY
0,"Aug 29, 2017 11:39:00 AM","Aug 28, 2017 3:34:35 PM","Aug 28, 2017 3:40:31 PM",,daniel.anastos@komaza.org,356,,Duel14,N,,,G,1,(Br),0,0,0,0,0,0,0,1,,C,1,,,C,0,0,0,0,0,0,0,1,,NB,0,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:0b4cf03b-3e17-49b0-955b-4a75e8a84703,1708231724,uuid:0b4cf03b-3e17-49b0-955b-4a75e8a84703
1,"Sep 12, 2017 6:36:47 PM","Sep 12, 2017 10:56:50 AM","Sep 12, 2017 1:00:20 PM",,demo,5707,,Lash11,clearfell,no,,good_road,yes,no_obstacles,0,0,0,0,0,1,0,0,,clay,no,,,none,0,0,0,0,1,0,0,0,,no_bushes,no,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e,1709121045,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e
2,"Sep 12, 2017 6:36:47 PM","Sep 12, 2017 12:30:39 PM","Sep 12, 2017 12:46:52 PM",,demo,973,,Lash10,normal,no,,good_road,yes,no_obstacles,0,0,0,0,0,1,0,0,,clay,yes,,,none,0,0,0,0,1,0,0,0,,no_bushes,no,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c,1709121045,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c
3,"Sep 12, 2017 6:36:48 PM","Sep 12, 2017 1:18:34 PM","Sep 12, 2017 1:26:13 PM",,demo,459,,Lash6,clearfell,no,,good_road,yes,no_obstacles,0,0,0,0,0,1,0,0,,clay,no,,,none,0,0,0,0,1,0,0,0,,no_bushes,no,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77,1709121045,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77
4,"Sep 12, 2017 6:56:23 PM","Sep 12, 2017 10:51:21 AM","Sep 12, 2017 11:27:20 AM",,demo,2159,,lash20,clearfell,no,,good_road,yes,sand,0,0,0,1,0,0,0,0,,clay,no,,,crops,0,1,0,0,0,0,0,0,,no_bushes,no,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7,1709121045,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7


### # This dataset don't have a fiedd for tree_count, means more processing needs to be done to figure that out


In [332]:
# Filter columns
#submissiondate	farmid	username	dbh
filter_col = [col for col in df_v2 if col == 'SubmissionDate' or col == 'username' or col == 'uniqueID' or col == 'KEY' or col.startswith('dbh_')]
filter_col

['SubmissionDate',
 'username',
 'uniqueID',
 'dbh_1',
 'dbh_2',
 'dbh_3',
 'dbh_4',
 'dbh_5',
 'dbh_6',
 'dbh_7',
 'dbh_8',
 'dbh_9',
 'dbh_10',
 'dbh_11',
 'dbh_12',
 'dbh_13',
 'dbh_14',
 'dbh_15',
 'dbh_16',
 'dbh_17',
 'dbh_18',
 'dbh_19',
 'dbh_20',
 'dbh_21',
 'dbh_22',
 'dbh_23',
 'dbh_24',
 'dbh_25',
 'dbh_26',
 'dbh_27',
 'dbh_28',
 'dbh_29',
 'dbh_30',
 'dbh_31',
 'dbh_32',
 'dbh_33',
 'dbh_34',
 'dbh_35',
 'dbh_36',
 'dbh_37',
 'dbh_38',
 'dbh_39',
 'dbh_40',
 'dbh_41',
 'dbh_42',
 'dbh_43',
 'dbh_44',
 'dbh_45',
 'dbh_46',
 'dbh_47',
 'dbh_48',
 'dbh_49',
 'dbh_50',
 'dbh_51',
 'dbh_52',
 'dbh_53',
 'dbh_54',
 'dbh_55',
 'dbh_56',
 'dbh_57',
 'dbh_58',
 'dbh_59',
 'dbh_60',
 'dbh_61',
 'dbh_62',
 'dbh_63',
 'dbh_64',
 'dbh_65',
 'dbh_66',
 'dbh_67',
 'dbh_68',
 'dbh_69',
 'dbh_70',
 'dbh_71',
 'dbh_72',
 'dbh_73',
 'dbh_74',
 'dbh_75',
 'dbh_76',
 'dbh_77',
 'dbh_78',
 'dbh_79',
 'dbh_80',
 'dbh_81',
 'dbh_82',
 'dbh_83',
 'dbh_84',
 'dbh_85',
 'dbh_86',
 'dbh_87',
 'dbh_8

In [333]:
# Select only the filtered columns
df_2_filtered = df_v2[filter_col]

# Preview the new dataset
df_2_filtered.head()

,SubmissionDate,username,uniqueID,dbh_1,dbh_2,dbh_3,dbh_4,dbh_5,dbh_6,dbh_7,dbh_8,dbh_9,dbh_10,dbh_11,dbh_12,dbh_13,dbh_14,dbh_15,dbh_16,dbh_17,dbh_18,dbh_19,dbh_20,dbh_21,dbh_22,dbh_23,dbh_24,dbh_25,dbh_26,dbh_27,dbh_28,dbh_29,dbh_30,dbh_31,dbh_32,dbh_33,dbh_34,dbh_35,dbh_36,dbh_37,...,dbh_262,dbh_263,dbh_264,dbh_265,dbh_266,dbh_267,dbh_268,dbh_269,dbh_270,dbh_271,dbh_272,dbh_273,dbh_274,dbh_275,dbh_276,dbh_277,dbh_278,dbh_279,dbh_280,dbh_281,dbh_282,dbh_283,dbh_284,dbh_285,dbh_286,dbh_287,dbh_288,dbh_289,dbh_290,dbh_291,dbh_292,dbh_293,dbh_294,dbh_295,dbh_296,dbh_297,dbh_298,dbh_299,dbh_300,KEY
0,"Aug 29, 2017 11:39:00 AM",daniel.anastos@komaza.org,Duel14,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:0b4cf03b-3e17-49b0-955b-4a75e8a84703
1,"Sep 12, 2017 6:36:47 PM",demo,Lash11,3.3,3.2,3.5,2.3,2.3,2.4,2.6,2,2.6,1.6,2.5,2,3.2,2.8,3.1,2.4,3.1,3.2,2.6,2.8,3.3,3,2.8,2.6,2.9,3.9,3,2.6,3.1,3.5,2.6,3.4,2.6,3.1,3.4,2.7,3,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e
2,"Sep 12, 2017 6:36:47 PM",demo,Lash10,2,2.5,2.5,2.5,2.3,2.9,1.8,1.6,2.3,2.3,3,3.2,2.4,2.5,2.7,3.5,2.6,2.8,4,3.8,1.6,1.3,2,1.9,2,1.7,1.5,2.4,2.1,2.3,2,2.5,2.6,1.5,2.5,2.6,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c
3,"Sep 12, 2017 6:36:48 PM",demo,Lash6,3.5,3.1,4.1,4.3,4.8,2.1,2,2.8,3,2.6,2,1.8,3,2.7,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77
4,"Sep 12, 2017 6:56:23 PM",demo,lash20,2.5,2.4,1.6,2.4,2,2.7,3.4,3.1,1.2,2.5,2.2,2.2,3.5,2.3,2.1,3.2,2.1,3.5,3,3.2,2.2,3.6,2.9,3.1,2.4,3,3.4,2.2,2.3,2.5,2.1,3.1,2.5,1.8,1.2,2.6,1.4,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7


In [334]:
# Melt down the dataframe from wide to long format

df_2_melted = df_2_filtered.melt(
    id_vars = ['SubmissionDate', 'username', 'uniqueID', 'KEY'],
    var_name = 'tree_dbh',
    value_name = 'dbh_value'
  )

# Preview the melted df
df_2_melted.head()

,SubmissionDate,username,uniqueID,KEY,tree_dbh,dbh_value
0,"Aug 29, 2017 11:39:00 AM",daniel.anastos@komaza.org,Duel14,uuid:0b4cf03b-3e17-49b0-955b-4a75e8a84703,dbh_1,
1,"Sep 12, 2017 6:36:47 PM",demo,Lash11,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e,dbh_1,3.3
2,"Sep 12, 2017 6:36:47 PM",demo,Lash10,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c,dbh_1,2
3,"Sep 12, 2017 6:36:48 PM",demo,Lash6,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77,dbh_1,3.5
4,"Sep 12, 2017 6:56:23 PM",demo,lash20,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7,dbh_1,2.5


In [335]:
# check datatypes
df_2_melted.dtypes

SubmissionDate    object
username          object
uniqueID          object
KEY               object
tree_dbh          object
dbh_value         object
dtype: object

In [336]:
# Convert data types tp correct formats
df_2_melted.SubmissionDate = pd.to_datetime(df_2_melted.SubmissionDate)
df_2_melted.dbh_value = pd.to_numeric(df_2_melted.dbh_value)

# Preview dtypes
df_2_melted.dtypes

SubmissionDate    datetime64[ns]
username                  object
uniqueID                  object
KEY                       object
tree_dbh                  object
dbh_value                float64
dtype: object

In [337]:
# Preview data frame
df_2_melted.head()

,SubmissionDate,username,uniqueID,KEY,tree_dbh,dbh_value
0,2017-08-29 11:39:00,daniel.anastos@komaza.org,Duel14,uuid:0b4cf03b-3e17-49b0-955b-4a75e8a84703,dbh_1,NaN
1,2017-09-12 18:36:47,demo,Lash11,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e,dbh_1,3.3
2,2017-09-12 18:36:47,demo,Lash10,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c,dbh_1,2.0
3,2017-09-12 18:36:48,demo,Lash6,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77,dbh_1,3.5
4,2017-09-12 18:56:23,demo,lash20,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7,dbh_1,2.5


In [338]:
# filter only rows with dbh value
df_2_melted = df_2_melted[df_2_melted['dbh_value'].notna()]

# Preview
df_2_melted.head()

,SubmissionDate,username,uniqueID,KEY,tree_dbh,dbh_value
1,2017-09-12 18:36:47,demo,Lash11,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e,dbh_1,3.3
2,2017-09-12 18:36:47,demo,Lash10,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c,dbh_1,2.0
3,2017-09-12 18:36:48,demo,Lash6,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77,dbh_1,3.5
4,2017-09-12 18:56:23,demo,lash20,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7,dbh_1,2.5
5,2017-09-12 18:56:24,demo,Lash9,uuid:395afe65-c47c-4f93-8e24-2c87f9539626,dbh_1,2.6


In [339]:
# Aggregate the data by counting the dbh values
df_2_aggr = df_2_melted.pivot_table(index=['SubmissionDate', 'username', 'uniqueID', 'KEY'], values=['dbh_value'], aggfunc='count')

# Preveiw the aggregated dataframe
df_2_aggr.head()

dbh_value
SubmissionDate      username uniqueID KEY                                                 
2017-09-12 18:36:47 demo     Lash10   uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c         36
                             Lash11   uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e        202
2017-09-12 18:36:48 demo     Lash6    uuid:42e9b676-19ea-409f-94fc-d891f08c0c77         14
2017-09-12 18:56:23 demo     lash20   uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7        151
2017-09-12 18:56:24 demo     Lash9    uuid:395afe65-c47c-4f93-8e24-2c87f9539626        219

In [340]:
# Reset index
df_2_aggr.reset_index(inplace=True)

# Preview dataset
df_2_aggr.head()

,SubmissionDate,username,uniqueID,KEY,dbh_value
0,2017-09-12 18:36:47,demo,Lash10,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c,36
1,2017-09-12 18:36:47,demo,Lash11,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e,202
2,2017-09-12 18:36:48,demo,Lash6,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77,14
3,2017-09-12 18:56:23,demo,lash20,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7,151
4,2017-09-12 18:56:24,demo,Lash9,uuid:395afe65-c47c-4f93-8e24-2c87f9539626,219


In [341]:
# Rearrange the column positions
df_2_final = df_2_aggr[['SubmissionDate', 'username', 'uniqueID', 'dbh_value', 'KEY']]

# Preview the new arranged dataset
df_2_final.head()

,SubmissionDate,username,uniqueID,dbh_value,KEY
0,2017-09-12 18:36:47,demo,Lash10,36,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c
1,2017-09-12 18:36:47,demo,Lash11,202,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e
2,2017-09-12 18:36:48,demo,Lash6,14,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77
3,2017-09-12 18:56:23,demo,lash20,151,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7
4,2017-09-12 18:56:24,demo,Lash9,219,uuid:395afe65-c47c-4f93-8e24-2c87f9539626


In [342]:
# Rename columns to match prior datasets
df_2_final.rename(columns={'SubmissionDate': 'date', 'uniqueID': 'farmid', 'dbh_value': 'tree_count', 'KEY': 'primary_key'}, inplace=True)

# Preview the changes column names
df_2_final.head()

,date,username,farmid,tree_count,primary_key
0,2017-09-12 18:36:47,demo,Lash10,36,uuid:5e81e5e5-5853-4e80-a6ab-fd41d78bdc7c
1,2017-09-12 18:36:47,demo,Lash11,202,uuid:f2ec2efd-1bca-440b-ac4b-3a0ce2756d7e
2,2017-09-12 18:36:48,demo,Lash6,14,uuid:42e9b676-19ea-409f-94fc-d891f08c0c77
3,2017-09-12 18:56:23,demo,lash20,151,uuid:7ee6c885-0a08-465f-98e0-f06f0498d1b7
4,2017-09-12 18:56:24,demo,Lash9,219,uuid:395afe65-c47c-4f93-8e24-2c87f9539626


### Evaluation Report_v5_WIDE

In [343]:
# import Evaluation Report v3_WIDE
sh = gc.open('Evaluation Report_v5_WIDE')
# Select Spreadsheet
# By title
worksheet_v5 = sh.worksheet('Evaluation Report_v5_WIDE')
data_v5 = worksheet_v5.get_all_values()
headers = data_v5.pop(0)

df_v5 = pd.DataFrame(data_v5, columns=headers)

# Preview dataframe
df_v5.head()

,SubmissionDate,starttime,endtime,username,duration,team,uniqueID,id,farmername,sublocation,species,farmerphone,nickname,caretakername,plantingyear,count,confirm,mismatch,operation,years_away,similarfarms,uniqueidsimilarfarms,roadgoodorbad,Roadwidth,bridgehills,bridgehills_bridge,bridgehills_hills,bridgehills_narrow,bridgehills_sand,bridgehills_crops,bridgehills_no_obstacles,bridgehills_other,other_obstacles_main_road,soiltype,terrainyesno,bigsmallhills,travelupordown,obstaclestowoodlot,obstaclestowoodlot_bush_roots_stumps,obstaclestowoodlot_crops,...,tree_number_6,height_6,tree_number_7,height_7,tree_number_8,height_8,tree_number_9,height_9,tree_number_10,height_10,tree_number_11,height_11,tree_number_12,height_12,tree_number_13,height_13,tree_number_14,height_14,tree_number_15,height_15,tree_number_16,height_16,tree_number_17,height_17,tree_number_18,height_18,tree_number_19,height_19,tree_number_20,height_20,comment,signee,representative,signe,phone,didntsign,othereason,instanceID,formdef_version,KEY
0,"May 7, 2019 4:25:51 PM","May 7, 2019 2:45:40 PM","May 7, 2019 4:25:16 PM",samson.ngoba@komaza.org,2177,team_a,alna28,,,,,,,,,,not_correct,CTO does not pull data.,clearfell,,yes,ana34,,,,,,,,,,,,,,,,,,,...,2,12.2,7,12.8,8,14.2,9,11.9,10,14.8,11,15.2,12,16.2,13,12.4,14,12.6,15,11.3,16,10.9,17,12.8,18,15.2,19,15.6,20,16.2,Trees were by fire.,none,,,,,,uuid:084b2101-a90f-44ce-984a-0e472bb0244c,1901211139,uuid:084b2101-a90f-44ce-984a-0e472bb0244c
1,"May 7, 2019 4:26:57 PM","May 7, 2019 3:46:14 PM","May 7, 2019 4:26:39 PM",samson.ngoba@komaza.org,1973,team_a,alna34,,,,,,,,,,not_correct,CTO can not pull data,clearfell,,no,,bad_road,no_road,sand,0,0,0,1,0,0,0,,loam,terrain_not_flat,small,TravelDown,river,0,0,...,6,11.9,7,12.8,8,15,9,14.5,10,15,11,14.8,12,10.9,13,16.2,14,12.9,15,16.2,16,9.9,17,12.1,18,13.9,19,12.8,20,15.3,Tress were burnt by fire,none,,,,,,uuid:58c0c06f-f7e0-4616-b751-7f5b1a2e1910,1901211139,uuid:58c0c06f-f7e0-4616-b751-7f5b1a2e1910
2,"May 8, 2019 3:01:52 PM","May 8, 2019 2:38:19 PM","May 8, 2019 3:01:21 PM",samson.ngoba@komaza.org,1382,team_a,stse221,,,,,,,,,,not_correct,CTO can't pull data,clearfell,,no,,bad_road,road_not_wide,hills,0,1,0,0,0,0,0,,clay,terrain_not_flat,small,TravelDown,bush_roots_stumps,1,0,...,6,14.9,7,14.2,8,14.8,9,12,10,16,11,15.2,12,9,13,10.9,14,14.2,15,13.6,16,11,17,13.3,18,6,19,10,20,12.7,,none,,,,,,uuid:de95ea8a-71af-44ac-8f99-afc401b7323d,1901211139,uuid:de95ea8a-71af-44ac-8f99-afc401b7323d
3,"Aug 8, 2019 11:07:49 AM","Jun 26, 2019 3:04:48 PM","Jun 26, 2019 3:10:51 PM",samson.ngoba@komaza.org,226,team_a,jiel132,,,,,,,,,,correct,,normal,,yes,Fff,bad_road,road_not_wide,sand,0,0,0,1,0,0,0,,loam,terrain_not_flat,small,TravelDown,crops,0,1,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Ff,none,,,,,,uuid:1baf636c-6d55-49a8-8230-b899cbca354d,1901211139,uuid:1baf636c-6d55-49a8-8230-b899cbca354d


In [344]:
# Get a subset of the dataset
df_v5_filter = df_v5[['SubmissionDate', 'username', 'uniqueID', 'tree_count', 'KEY']]

# Preview the filter
df_v5_filter.head()

,SubmissionDate,username,uniqueID,tree_count,KEY
0,"May 7, 2019 4:25:51 PM",samson.ngoba@komaza.org,alna28,171,uuid:084b2101-a90f-44ce-984a-0e472bb0244c
1,"May 7, 2019 4:26:57 PM",samson.ngoba@komaza.org,alna34,170,uuid:58c0c06f-f7e0-4616-b751-7f5b1a2e1910
2,"May 8, 2019 3:01:52 PM",samson.ngoba@komaza.org,stse221,47,uuid:de95ea8a-71af-44ac-8f99-afc401b7323d
3,"Aug 8, 2019 11:07:49 AM",samson.ngoba@komaza.org,jiel132,22,uuid:1baf636c-6d55-49a8-8230-b899cbca354d


In [345]:
# Rename the columns
df_v5_filter.rename(columns={'SubmissionDate': 'date', 'uniqueID': 'farmid', 'KEY': 'primary_key'}, inplace=True)

# Preview the dataframe
df_v5_filter.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,username,farmid,tree_count,primary_key
0,"May 7, 2019 4:25:51 PM",samson.ngoba@komaza.org,alna28,171,uuid:084b2101-a90f-44ce-984a-0e472bb0244c
1,"May 7, 2019 4:26:57 PM",samson.ngoba@komaza.org,alna34,170,uuid:58c0c06f-f7e0-4616-b751-7f5b1a2e1910
2,"May 8, 2019 3:01:52 PM",samson.ngoba@komaza.org,stse221,47,uuid:de95ea8a-71af-44ac-8f99-afc401b7323d
3,"Aug 8, 2019 11:07:49 AM",samson.ngoba@komaza.org,jiel132,22,uuid:1baf636c-6d55-49a8-8230-b899cbca354d


In [346]:
# Check the data types of the dataset
df_v5_filter.dtypes

date           object
username       object
farmid         object
tree_count     object
primary_key    object
dtype: object

In [347]:
# cast the data types to correct formats

df_v5_filter.date = pd.to_datetime(df_v5_filter.date)
df_v5_filter.tree_count = pd.to_numeric(df_v5_filter.tree_count)

# Preview dtypes
df_v5_filter.dtypes

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


date           datetime64[ns]
username               object
farmid                 object
tree_count              int64
primary_key            object
dtype: object

In [348]:
# Preview the dataframe
df_v5_filter.head()

,date,username,farmid,tree_count,primary_key
0,2019-05-07 16:25:51,samson.ngoba@komaza.org,alna28,171,uuid:084b2101-a90f-44ce-984a-0e472bb0244c
1,2019-05-07 16:26:57,samson.ngoba@komaza.org,alna34,170,uuid:58c0c06f-f7e0-4616-b751-7f5b1a2e1910
2,2019-05-08 15:01:52,samson.ngoba@komaza.org,stse221,47,uuid:de95ea8a-71af-44ac-8f99-afc401b7323d
3,2019-08-08 11:07:49,samson.ngoba@komaza.org,jiel132,22,uuid:1baf636c-6d55-49a8-8230-b899cbca354d


### EvaluationCOMMs_Pilot_WIDE

In [349]:
# import Evaluation comms pilot WIDE
sh = gc.open('EvaluationCOMMs_Pilot_WIDE')
# Select Spreadsheet
# By title
worksheet_v6 = sh.worksheet('EvaluationCOMMs_Pilot_WIDE')
data_v6 = worksheet_v6.get_all_values()
headers = data_v6.pop(0)

df_v6 = pd.DataFrame(data_v6, columns=headers)

# Preview dataframe
df_v6.head()

,SubmissionDate,starttime,endtime,username,duration,team,uniqueID,id,farmername,sublocation,species,farmerphone,nickname,caretakername,plantingyear,count,confirm,mismatch,operation,years_away,similarfarms,uniqueidsimilarfarms,roadgoodorbad,Roadwidth,bridgehills,bridgehills_bridge,bridgehills_hills,bridgehills_narrow,bridgehills_sand,bridgehills_crops,bridgehills_no_obstacles,bridgehills_other,other_obstacles_main_road,soiltype,terrainyesno,bigsmallhills,travelupordown,obstaclestowoodlot,obstaclestowoodlot_bush_roots_stumps,obstaclestowoodlot_crops,...,height_5,tree_number_6,height_6,tree_number_7,height_7,tree_number_8,height_8,tree_number_9,height_9,tree_number_10,height_10,tree_number_11,height_11,tree_number_12,height_12,tree_number_13,height_13,tree_number_14,height_14,tree_number_15,height_15,tree_number_16,height_16,tree_number_17,height_17,tree_number_18,height_18,tree_number_19,height_19,tree_number_20,height_20,comment,signee,representative,signe,didntsign,othereason,instanceID,formdef_version,KEY
0,"Nov 26, 2018 5:02:08 PM","Nov 26, 2018 9:10:06 AM","Nov 26, 2018 11:43:20 AM",eunice.gambo@komaza.org,2538,team_b,stse199,stse199,Dama ngumbao mwatembo,Dungicha/Muhoni,Eucalyptus,705264879,Hawe furaha,Farmer,2010,247,correct,,clearfell,,no,,good_road,road_not_wide,narrow,0,0,1,0,0,0,0,,sand,terrain_is_flat,,,bush_roots_stumps,1,0,...,15.9,6,15.8,7,15.5,8,13.3,9,13.4,10,14.4,11,14.6,12,13.7,13,12.9,14,15.2,15,14.6,16,15.5,17,15.7,18,14.9,19,12.7,20,13.5,,farmer,,yes,,,uuid:ee411bd4-40c3-4b26-b6c3-5f9dd54a2d2b,1811260808,uuid:ee411bd4-40c3-4b26-b6c3-5f9dd54a2d2b
1,"Nov 26, 2018 5:02:08 PM","Nov 26, 2018 11:57:16 AM","Nov 26, 2018 3:16:49 PM",eunice.gambo@komaza.org,2379,team_b,stse219,stse219,Julius kahindi kenga,Mweza/Migodomani,Eucalyptus,700259962,Dunda,Katana kenga,2013,53,correct,,clearfell,,no,,bad_road,no_road,hills other,0,1,0,0,0,0,1,Bush and stumps,sand,terrain_not_flat,small,up_and_down,bush_roots_stumps,1,0,...,12.7,6,14.5,7,13.4,8,12.6,9,13.6,10,13.2,11,11.7,12,12.8,13,12.2,14,11.4,15,11.6,16,12.1,17,11.3,18,11.4,19,11.5,20,11.4,,farmer,,yes,,,uuid:f3e1f78e-ff75-4093-9432-bc3df0ed9c86,1811260808,uuid:f3e1f78e-ff75-4093-9432-bc3df0ed9c86
2,"Nov 26, 2018 5:02:09 PM","Nov 26, 2018 1:41:11 PM","Nov 26, 2018 3:22:41 PM",eunice.gambo@komaza.org,1278,team_b,stse224,stse224,Julius kahindi kenga,Mweza/Migodomani,Eucalyptus,700259962,Dunda,Farmer,2010,189,correct,,clearfell,,no,,good_road,road_not_wide,narrow,0,0,1,0,0,0,0,,sand,terrain_is_flat,,,bush_roots_stumps,1,0,...,14.1,6,13.7,7,12.7,8,13.1,9,14.3,10,13.6,11,14.6,12,15,13,14.4,14,14.8,15,15.2,16,14.3,17,13.9,18,12.9,19,14.5,20,12.7,,farmer,,yes,,,uuid:1b653008-172e-4993-a0d1-509bcbddd1f2,1811260808,uuid:1b653008-172e-4993-a0d1-509bcbddd1f2
3,"Nov 27, 2018 6:05:06 PM","Nov 27, 2018 8:18:51 AM","Nov 27, 2018 10:30:16 AM",eunice.gambo@komaza.org,2258,team_b,stse225,stse225,Chadi kalume karisa,Mweza/Migodomani,Eucalyptus,790202862,Chadi,Farmer,2011,66,correct,,clearfell,,yes,stse224,,,,,,,,,,,,,,,,,,,...,14.1,6,13.8,7,13.1,8,14.3,9,12.7,10,14.5,11,15.1,12,13.7,13,14.4,14,12.7,15,12.9,16,13.8,17,14.4,18,14.2,19,13.7,20,14.5,None,farmer,,yes,,,uuid:5d9db66a-3b6b-4658-b1f8-a8f2f93caaa8,1811262033,uuid:5d9db66a-3b6b-4658-b1f8-a8f2f93caaa8
4,"Nov 27, 2018 6:05:07 PM","Nov 27, 2018 10:35:46 AM","Nov 27, 2018 2:24:28 PM",eunice.gambo@komaza.org,2778,team_b,stse218,stse218,Kana kenga karisa,Mweza/Migodomani,Eucalyptus,704083049,Hawe sofia,Farmer,2010,108,correct,,clearfell,,yes,stse219,,,,,,,,,,,,,,,,,,,...,12.8,6,13.5,7,13.7,8,12.6,9,11.7,10,12.5,11,12.9,12,15.1,13,14.4,14,13.1,15,11.8,16,13.6,17,14.3,18,14.2,19,13.6,20,16.7,,farmer,,yes,,,uuid:8adff194-9aa6-41cb-a922-c44b63ad99fb,1811262033,uuid:8adff194-9aa6-41cb-a922-c44b63ad99fb


In [350]:
# Get a subset of the dataset
df_v6_filter = df_v6[['SubmissionDate', 'username', 'uniqueID', 'tree_count', 'KEY']]

# Preview the filter
df_v6_filter.head()

,SubmissionDate,username,uniqueID,tree_count,KEY
0,"Nov 26, 2018 5:02:08 PM",eunice.gambo@komaza.org,stse199,115,uuid:ee411bd4-40c3-4b26-b6c3-5f9dd54a2d2b
1,"Nov 26, 2018 5:02:08 PM",eunice.gambo@komaza.org,stse219,45,uuid:f3e1f78e-ff75-4093-9432-bc3df0ed9c86
2,"Nov 26, 2018 5:02:09 PM",eunice.gambo@komaza.org,stse224,107,uuid:1b653008-172e-4993-a0d1-509bcbddd1f2
3,"Nov 27, 2018 6:05:06 PM",eunice.gambo@komaza.org,stse225,43,uuid:5d9db66a-3b6b-4658-b1f8-a8f2f93caaa8
4,"Nov 27, 2018 6:05:07 PM",eunice.gambo@komaza.org,stse218,83,uuid:8adff194-9aa6-41cb-a922-c44b63ad99fb


In [351]:
# Rename the columns
df_v6_filter.rename(columns={'SubmissionDate': 'date', 'uniqueID': 'farmid', 'KEY': 'primary_key'}, inplace=True)

# Preview the dataframe
df_v6_filter.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,username,farmid,tree_count,primary_key
0,"Nov 26, 2018 5:02:08 PM",eunice.gambo@komaza.org,stse199,115,uuid:ee411bd4-40c3-4b26-b6c3-5f9dd54a2d2b
1,"Nov 26, 2018 5:02:08 PM",eunice.gambo@komaza.org,stse219,45,uuid:f3e1f78e-ff75-4093-9432-bc3df0ed9c86
2,"Nov 26, 2018 5:02:09 PM",eunice.gambo@komaza.org,stse224,107,uuid:1b653008-172e-4993-a0d1-509bcbddd1f2
3,"Nov 27, 2018 6:05:06 PM",eunice.gambo@komaza.org,stse225,43,uuid:5d9db66a-3b6b-4658-b1f8-a8f2f93caaa8
4,"Nov 27, 2018 6:05:07 PM",eunice.gambo@komaza.org,stse218,83,uuid:8adff194-9aa6-41cb-a922-c44b63ad99fb


In [352]:
# Check the data types of the dataset
df_v6_filter.dtypes

date           object
username       object
farmid         object
tree_count     object
primary_key    object
dtype: object

In [353]:
# cast the data types to correct formats

df_v6_filter.date = pd.to_datetime(df_v6_filter.date)
df_v6_filter.tree_count = pd.to_numeric(df_v6_filter.tree_count)

# Preview dtypes
df_v6_filter.dtypes

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


date           datetime64[ns]
username               object
farmid                 object
tree_count            float64
primary_key            object
dtype: object

In [354]:
# Preview the dataframe
df_v6_filter.head()

,date,username,farmid,tree_count,primary_key
0,2018-11-26 17:02:08,eunice.gambo@komaza.org,stse199,115.0,uuid:ee411bd4-40c3-4b26-b6c3-5f9dd54a2d2b
1,2018-11-26 17:02:08,eunice.gambo@komaza.org,stse219,45.0,uuid:f3e1f78e-ff75-4093-9432-bc3df0ed9c86
2,2018-11-26 17:02:09,eunice.gambo@komaza.org,stse224,107.0,uuid:1b653008-172e-4993-a0d1-509bcbddd1f2
3,2018-11-27 18:05:06,eunice.gambo@komaza.org,stse225,43.0,uuid:5d9db66a-3b6b-4658-b1f8-a8f2f93caaa8
4,2018-11-27 18:05:07,eunice.gambo@komaza.org,stse218,83.0,uuid:8adff194-9aa6-41cb-a922-c44b63ad99fb


### Marking and Evaluation Raw 2.0
This represent the Live dataset that is current in production environment and used regularly by the marking and evaluation team

In [355]:
# import Evaluation comms pilot WIDE
sh = gc.open('Marking and Evaluation Raw 2.0')
# Select Spreadsheet
# By title
worksheet_v7 = sh.worksheet('data')
data_v7 = worksheet_v7.get_all_values()
headers = data_v7.pop(0)

df_v7 = pd.DataFrame(data_v7, columns=headers)

# Preview dataframe
df_v7.head()

,SubmissionDate,starttime,endtime,username,duration,kcode,sublocation_id,tree_species_id,planting_year_id,survived_trees,confirm,mismatch,operation,specialcase,tree_count,rotten,stolen,classtotal_2,classtotal_3,classtotal_4,classtotal_5,classtotal_6,classtotal_7,classtotal_8,classtotal_9,classtotal_10,classtotal_11,classtotal_12,classtotal_13,classtotal_14,classtotal_15,total_trees,total_cash,total_volume,extractn_dist,signee,representative,signed,didntsign,other_reason,mpesa_phone_no,mpesa_phone_no_owner,mpesa_phone_name,comment,instanceID,KEY,survived_trees,other_reason,mpesa_phone_no,mpesa_phone_no_owner,mpesa_phone_name
0,2019-08-14T13:54:13.784Z,2019-08-14T10:40:57.028+03,2019-08-14T16:44:14.403+03,demo,4470,klf-2009-0010,Petanguo,Eucalyptus Grandis,2009,200,correct,,finalharvest,,128,5,14,0,25,52,22,12,6,4,2,0,0,0,0,0,0,123,23600,8.408745683,71.15938291,farmer,,,,,717455494,farmer,,,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa,,,,,
1,2019-08-14T13:51:03.457Z,2019-08-14T15:07:32.591+03,2019-08-14T15:28:13.059+03,johnson.ngumbao@komaza.org,596,klf-2010-0215,Petanguo,Eucalyptus Grandis,2010,320,not_abletoconfirm,,special,The farmer is not recognized at the area. The ...,,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,,,,,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49,,,,,
2,2019-08-14T13:54:04.777Z,2019-08-14T09:46:59.358+03,2019-08-14T16:49:21.161+03,demo,3241,klf-2009-0126,Petanguo,Eucalyptus Grandis,2009,200,correct,,finalharvest,,56,0,3,0,1,16,13,15,5,3,0,3,0,0,0,0,0,56,14620,5.813401612,13.13012609,representative,Relative,,,,708235267,authorisedrep,Elizabeth Dama Simion,,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308,,,,,
3,2019-08-14T13:50:18.188Z,2019-08-14T11:42:42.228+03,2019-08-14T15:06:17.162+03,johnson.ngumbao@komaza.org,1465,klf-2010-0162,Petanguo,Eucalyptus Grandis,2010,320,correct,,finalharvest,,85,0,89,1,5,32,25,11,2,5,3,1,0,0,0,0,0,85,18990,7.248765833,371.530087,farmer,,,,,708236657,farmer,,No,uuid:76198455-a30c-4ffd-85ca-2512a53beabe,uuid:76198455-a30c-4ffd-85ca-2512a53beabe,,,,,
4,2019-08-14T13:50:04.485Z,2019-08-14T10:57:19.631+03,2019-08-14T14:59:51.245+03,johnson.ngumbao@komaza.org,788,klf-2010-0067,Petanguo,Eucalyptus Grandis,2010,320,correct,,finalharvest,,9,0,37,0,1,1,1,4,2,0,0,0,0,0,0,0,0,9,2410,0.9725529675,1178.428983,folowup_case,,,,,,,,No,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502,,,,,


In [356]:
# Get a subset of the dataset
df_v7_filter = df_v7[['SubmissionDate', 'username', 'kcode', 'total_trees', 'KEY']]

# Preview the filter
df_v7_filter.head()

,SubmissionDate,username,kcode,total_trees,KEY
0,2019-08-14T13:54:13.784Z,demo,klf-2009-0010,123,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa
1,2019-08-14T13:51:03.457Z,johnson.ngumbao@komaza.org,klf-2010-0215,0,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49
2,2019-08-14T13:54:04.777Z,demo,klf-2009-0126,56,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308
3,2019-08-14T13:50:18.188Z,johnson.ngumbao@komaza.org,klf-2010-0162,85,uuid:76198455-a30c-4ffd-85ca-2512a53beabe
4,2019-08-14T13:50:04.485Z,johnson.ngumbao@komaza.org,klf-2010-0067,9,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502


In [357]:
# Rename the columns
df_v7_filter.rename(columns={'SubmissionDate': 'date', 'KEY': 'primary_key', 'total_trees': 'tree_count'}, inplace=True)

# Preview the dataframe
df_v7_filter.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,username,kcode,tree_count,primary_key
0,2019-08-14T13:54:13.784Z,demo,klf-2009-0010,123,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa
1,2019-08-14T13:51:03.457Z,johnson.ngumbao@komaza.org,klf-2010-0215,0,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49
2,2019-08-14T13:54:04.777Z,demo,klf-2009-0126,56,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308
3,2019-08-14T13:50:18.188Z,johnson.ngumbao@komaza.org,klf-2010-0162,85,uuid:76198455-a30c-4ffd-85ca-2512a53beabe
4,2019-08-14T13:50:04.485Z,johnson.ngumbao@komaza.org,klf-2010-0067,9,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502


In [358]:
# Check the data types of the dataset
df_v7_filter.dtypes

date           object
username       object
kcode          object
tree_count     object
primary_key    object
dtype: object

In [359]:
# cast the data types to correct formats

df_v7_filter.date = pd.to_datetime(df_v7_filter.date)
df_v7_filter.tree_count = pd.to_numeric(df_v7_filter.tree_count)

# Preview dtypes
df_v7_filter.dtypes

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


date           datetime64[ns, UTC]
username                    object
kcode                       object
tree_count                   int64
primary_key                 object
dtype: object

In [360]:
# Remove time from date
df_v7_filter['date'] = pd.to_datetime(df_v7_filter['date']).dt.date

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [361]:
# Check new data types
df_v7_filter.dtypes

date           object
username       object
kcode          object
tree_count      int64
primary_key    object
dtype: object

In [362]:
# Change the date to datetime format
df_v7_filter.date = pd.to_datetime(df_v7_filter.date)

# Preview data types
df_v7_filter.dtypes

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


date           datetime64[ns]
username               object
kcode                  object
tree_count              int64
primary_key            object
dtype: object

In [363]:
# Preview the dataframe
df_v7_filter.head()

,date,username,kcode,tree_count,primary_key
0,2019-08-14,demo,klf-2009-0010,123,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa
1,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0215,0,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49
2,2019-08-14,demo,klf-2009-0126,56,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308
3,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0162,85,uuid:76198455-a30c-4ffd-85ca-2512a53beabe
4,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0067,9,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502


## Merging and Compiling

In [364]:
# Merge the historical MER first since they had similar structure
merged_historical = df_v3_filter.append(
    [df_v4_filter,df_2_final,df_v5_filter,df_v6_filter]
)

# Preview merged dataframe
merged_historical.tail()

,date,username,farmid,tree_count,primary_key
604,2019-08-02 14:53:53,johnson.ngumbao@komaza.org,emgr138,68.0,uuid:4f546fad-b913-4b02-aa9f-d0a558bd8e54
605,2019-08-02 14:53:53,johnson.ngumbao@komaza.org,lash129,0.0,uuid:2a33908f-5199-4751-9e78-eb22511c4dda
606,2019-08-02 18:29:54,rajab.abdallah@komaza.org,lash281,136.0,uuid:d41e3697-9049-4eea-8627-7ed25bb33018
607,2019-08-02 18:29:58,rajab.abdallah@komaza.org,lash282,0.0,uuid:cc801416-0489-4143-8d0b-d20c20efa2fa
608,2019-08-08 11:07:50,samson.ngoba@komaza.org,edvi346,20.0,uuid:d0d53d00-6e36-4dfa-959f-3ea3036dac37


In [365]:
# Reset index
merged_historical.reset_index(inplace=True)

# Preview
merged_historical.tail()

,index,date,username,farmid,tree_count,primary_key
2416,604,2019-08-02 14:53:53,johnson.ngumbao@komaza.org,emgr138,68.0,uuid:4f546fad-b913-4b02-aa9f-d0a558bd8e54
2417,605,2019-08-02 14:53:53,johnson.ngumbao@komaza.org,lash129,0.0,uuid:2a33908f-5199-4751-9e78-eb22511c4dda
2418,606,2019-08-02 18:29:54,rajab.abdallah@komaza.org,lash281,136.0,uuid:d41e3697-9049-4eea-8627-7ed25bb33018
2419,607,2019-08-02 18:29:58,rajab.abdallah@komaza.org,lash282,0.0,uuid:cc801416-0489-4143-8d0b-d20c20efa2fa
2420,608,2019-08-08 11:07:50,samson.ngoba@komaza.org,edvi346,20.0,uuid:d0d53d00-6e36-4dfa-959f-3ea3036dac37


## Get SSOT

In [366]:
# import Evaluation comms pilot WIDE
sh = gc.open('SSOT Shamba Person_MASTER')
# Select Spreadsheet
# By title
ssot = sh.worksheet('Shamba')
ssot_data = ssot.get_all_values()
headers = ssot_data.pop(0)

ssot_df = pd.DataFrame(ssot_data, columns=headers)

# Preview dataframe
ssot_df.head()

,kcode,region_id,polygon_id,location_id,sublocation_id,zone_id,homestead_location,fa_assigned_id,fo_assigned_id,fm_assigned_id,farmid,latitude,longitude,geopoint,farmer_official_name,common_name,availability_id,official_type_id,farmer_id,gender,contact_method_id,farmer_phone_number,hist_farmer_id,hist_farmer_name,landowner_id,cosignee_id,caretaker_name,caretaker_phone_number,tree_species_id,suitability,suitability_melia,suitability_eucalyptus,planting_year_id,planting_season_id,area_m2,shamba_size,tree_spacing,seedlings_planted,seedlings_planted_melia,seedlings_planted_eucalyptus,last_audit_date,last_audit_alive_trees,last_audit_survived_trees,shamba_status,ownership_status,contract_status,slo_url,contract_url,uuid,farmerfollowup_check
0,klf-2008-0001,kilifi,klf-2008-0001,Ngerenya,Ngerenya,zone-01-224,,Janet Sidi Nguma,Michael Mtengo Mundu,Alfred Mwatsuma Kiti,gase7,-3.524255766,39.87817724,point ( -3.524255766 39.87817724 ),Esther Kai Mwatunje,Mama Bandari,available_on_farm,national_id,id_2147628,Female,farmer,714232488,,,,id_2138432,,,Eucalyptus,Very suitable,Very suitable,Very suitable,2008,LR,2180,,2.5x2.5,200,,200,12-06-17 19:04,167,167,active,,,https://drive.google.com/open?id=1EjXIzpjYAVqn...,https://drive.google.com/open?id=1PuYE-Xzjq3Zi...,7397d896-bdaf-11e9-8473-631210d8f882,
1,klf-2008-0002,kilifi,klf-2008-0002,Ngerenya,Ngerenya,zone-01-224,,Janet Sidi Nguma,Michael Mtengo Mundu,Alfred Mwatsuma Kiti,gase8,-3.528535743,39.87702892,point ( -3.528535743 39.87702892 ),Pahe Kaingu Fondo,Mame Rehema,available_on_farm,national_id,id_9773938,Female,farmer,702374834,,,,id_27052844,,,Eucalyptus,Very suitable,Very suitable,Very suitable,2008,LR,1530,,2.5x2.5,200,,200,12-06-17 19:05,148,148,active,,,https://drive.google.com/open?id=15MONI3UOAGrc...,https://drive.google.com/open?id=17xNJM-wnSU2G...,7397d897-bdaf-11e9-8473-631210d8f882,
2,klf-2008-0003,kilifi,klf-2008-0003,Palakumi,Mariani/Vitsapuni,zone-01-208,,James Katana Karisa,Daidas Mwachiti,Tyson Sangura Wanyama,emgr206,-3.569655033,39.64606282,point ( -3.569655033 39.64606282 ),Kahindi Kazungu Kasingihi,Kahindi Kazungu,available_near_farm,national_id,id_9112244,Male,farmer,701694431,,,,id_11870882,,,Eucalyptus,Not suitable,Not suitable,Not suitable,2008,SR,190,,2.5x2.5,200,,200,01-08-17 20:20,30,40,active,,,https://drive.google.com/open?id=15bQ5W2MvZgqd...,https://drive.google.com/open?id=1E8v57fDm5-bK...,7397d898-bdaf-11e9-8473-631210d8f882,
3,klf-2008-0004,kilifi,klf-2008-0004,Palakumi,Mariani/Vitsapuni,zone-01-128,kwa kiraga kaviha,Christine Nzai Kalisho,James Mwahunga Ngumo,Mathias Mwabaya Kalama,emgr467,-3.607335748,39.64600114,point ( -3.607335748 39.64600114 ),Kahaso Kiraga Kaviha,Mkaza Binzai,available_near_farm,national_id,id_31487899,Female,other_family_member,799474601,,,,id_25205678,,,Eucalyptus,Not suitable,Not suitable,Not suitable,2008,SR,1190,,2.5x2.5,200,,200,05-12-17 22:00,74,158,active,,,https://drive.google.com/open?id=1IhlatEOLpM8g...,https://drive.google.com/open?id=1FjGwBZeeqOB_...,7397d899-bdaf-11e9-8473-631210d8f882,
4,klf-2008-0005,kilifi,klf-2008-0005,Ngerenya,Ngerenya,zone-01-224,,Janet Sidi Nguma,Michael Mtengo Mundu,Alfred Mwatsuma Kiti,gase4,-3.52362521,39.88113789,point ( -3.52362521 39.88113789 ),Joyce Dume Kadzenga,Mamake Maki,available_on_farm,national_id,id_13484471,Female,close_relative,79964251,,,,id_8459829,,,Eucalyptus,Very suitable,Very suitable,Very suitable,2008,LR,1680,,2.5x2.5,200,,200,13-06-17 9:06,86,107,active,,,https://drive.google.com/open?id=14dTHaTSJXTj-...,https://drive.google.com/open?id=1AwSCni4601Rp...,7397d89a-bdaf-11e9-8473-631210d8f882,


In [367]:
# Get a subset of the dataset
ssot_filter = ssot_df[['kcode', 'farmid']]

# Preview the filter
ssot_filter.head()

,kcode,farmid
0,klf-2008-0001,gase7
1,klf-2008-0002,gase8
2,klf-2008-0003,emgr206
3,klf-2008-0004,emgr467
4,klf-2008-0005,gase4


### Join the Historical Data with SSOT to get KCODE

In [368]:
ssot_filter.dtypes

kcode     object
farmid    object
dtype: object

In [369]:
merged_historical.drop(columns=['index'], inplace=True)

merged_historical.head()

,date,username,farmid,tree_count,primary_key
0,2018-02-01 10:16:29,samson.ngoba@komaza.org,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,2018-02-01 17:27:13,eunice.gambo@komaza.org,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,2018-02-01 17:27:14,eunice.gambo@komaza.org,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,2018-02-01 17:27:15,eunice.gambo@komaza.org,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,2018-02-02 07:00:07,samson.ngoba@komaza.org,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


In [370]:
merged_historical

,date,username,farmid,tree_count,primary_key
0,2018-02-01 10:16:29,samson.ngoba@komaza.org,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,2018-02-01 17:27:13,eunice.gambo@komaza.org,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,2018-02-01 17:27:14,eunice.gambo@komaza.org,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,2018-02-01 17:27:15,eunice.gambo@komaza.org,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,2018-02-02 07:00:07,samson.ngoba@komaza.org,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa
...,...,...,...,...,...
2416,2019-08-02 14:53:53,johnson.ngumbao@komaza.org,emgr138,68.0,uuid:4f546fad-b913-4b02-aa9f-d0a558bd8e54
2417,2019-08-02 14:53:53,johnson.ngumbao@komaza.org,lash129,0.0,uuid:2a33908f-5199-4751-9e78-eb22511c4dda
2418,2019-08-02 18:29:54,rajab.abdallah@komaza.org,lash281,136.0,uuid:d41e3697-9049-4eea-8627-7ed25bb33018
2419,2019-08-02 18:29:58,rajab.abdallah@komaza.org,lash282,0.0,uuid:cc801416-0489-4143-8d0b-d20c20efa2fa


In [371]:
# Ensure the farmid column is in lower case
merged_historical['farmid'] = merged_historical['farmid'].str.lower()
  
merged_historical.head()

,date,username,farmid,tree_count,primary_key
0,2018-02-01 10:16:29,samson.ngoba@komaza.org,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,2018-02-01 17:27:13,eunice.gambo@komaza.org,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,2018-02-01 17:27:14,eunice.gambo@komaza.org,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,2018-02-01 17:27:15,eunice.gambo@komaza.org,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,2018-02-02 07:00:07,samson.ngoba@komaza.org,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


In [372]:
merged_historical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2421 entries, 0 to 2420
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         2421 non-null   datetime64[ns]
 1   username     2421 non-null   object        
 2   farmid       2421 non-null   object        
 3   tree_count   1767 non-null   float64       
 4   primary_key  2421 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 94.7+ KB


In [373]:
# Ensure the farmid in ssot is in lower case
ssot_filter['farmid'] = ssot_filter['farmid'].str.lower()

ssot_filter.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,kcode,farmid
0,klf-2008-0001,gase7
1,klf-2008-0002,gase8
2,klf-2008-0003,emgr206
3,klf-2008-0004,emgr467
4,klf-2008-0005,gase4


In [374]:
ssot_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19683 entries, 0 to 19682
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   kcode   19683 non-null  object
 1   farmid  19683 non-null  object
dtypes: object(2)
memory usage: 307.7+ KB


In [375]:
joined_historical = pd.merge(left=merged_historical, right=ssot_filter, left_on='farmid', right_on='farmid', how='left')

#Preview
joined_historical.head()

,date,username,farmid,tree_count,primary_key,kcode
0,2018-02-01 10:16:29,samson.ngoba@komaza.org,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6,klf-2012-1388
1,2018-02-01 17:27:13,eunice.gambo@komaza.org,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f,klf-2012-1063
2,2018-02-01 17:27:14,eunice.gambo@komaza.org,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c,klf-2011-0676
3,2018-02-01 17:27:15,eunice.gambo@komaza.org,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf,klf-2012-0371
4,2018-02-02 07:00:07,samson.ngoba@komaza.org,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa,klf-2013-0123


In [376]:
joined_historical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2421 entries, 0 to 2420
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         2421 non-null   datetime64[ns]
 1   username     2421 non-null   object        
 2   farmid       2421 non-null   object        
 3   tree_count   1767 non-null   float64       
 4   primary_key  2421 non-null   object        
 5   kcode        2376 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 132.4+ KB


In [377]:
# Remove time from date

joined_historical['date'] = pd.to_datetime(joined_historical['date']).dt.date

# Preview the new data types
joined_historical.dtypes

date            object
username        object
farmid          object
tree_count     float64
primary_key     object
kcode           object
dtype: object

In [378]:
# Preview the dataset
joined_historical.head()

,date,username,farmid,tree_count,primary_key,kcode
0,2018-02-01,samson.ngoba@komaza.org,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6,klf-2012-1388
1,2018-02-01,eunice.gambo@komaza.org,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f,klf-2012-1063
2,2018-02-01,eunice.gambo@komaza.org,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c,klf-2011-0676
3,2018-02-01,eunice.gambo@komaza.org,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf,klf-2012-0371
4,2018-02-02,samson.ngoba@komaza.org,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa,klf-2013-0123


In [379]:
# change the date format
joined_historical.date = pd.to_datetime(joined_historical.date)

# Preview the new data types
joined_historical.dtypes

date           datetime64[ns]
username               object
farmid                 object
tree_count            float64
primary_key            object
kcode                  object
dtype: object

In [380]:
# Rearrange the columns
joined_historical = joined_historical[['date', 'username', 'kcode', 'farmid', 'tree_count', 'primary_key']]

# Preview the new data frame
joined_historical.head()

,date,username,kcode,farmid,tree_count,primary_key
0,2018-02-01,samson.ngoba@komaza.org,klf-2012-1388,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,2018-02-01,eunice.gambo@komaza.org,klf-2012-1063,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,2018-02-01,eunice.gambo@komaza.org,klf-2011-0676,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,2018-02-01,eunice.gambo@komaza.org,klf-2012-0371,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,2018-02-02,samson.ngoba@komaza.org,klf-2013-0123,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


### Reformat the Marking and Evaluation 2.0 to match Historical

In [381]:
# Preview the dataset for current live mer
df_v7_filter.head()

,date,username,kcode,tree_count,primary_key
0,2019-08-14,demo,klf-2009-0010,123,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa
1,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0215,0,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49
2,2019-08-14,demo,klf-2009-0126,56,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308
3,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0162,85,uuid:76198455-a30c-4ffd-85ca-2512a53beabe
4,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0067,9,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502


In [382]:
# Convert the kcode to lower
df_v7_filter['kcode'] = df_v7_filter['kcode'].str.lower()

df_v7_filter.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,username,kcode,tree_count,primary_key
0,2019-08-14,demo,klf-2009-0010,123,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa
1,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0215,0,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49
2,2019-08-14,demo,klf-2009-0126,56,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308
3,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0162,85,uuid:76198455-a30c-4ffd-85ca-2512a53beabe
4,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0067,9,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502


In [383]:
# Merge the current mer with SSOT to get the farmid
joined_current = pd.merge(left=df_v7_filter, right=ssot_filter, left_on='kcode', right_on='kcode', how='left')

#Preview
joined_current.head()

,date,username,kcode,tree_count,primary_key,farmid
0,2019-08-14,demo,klf-2009-0010,123,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa,crfa345
1,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0215,0,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49,stse143
2,2019-08-14,demo,klf-2009-0126,56,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308,crfa348
3,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0162,85,uuid:76198455-a30c-4ffd-85ca-2512a53beabe,crfa321
4,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0067,9,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502,crfa335


In [384]:
# Rearrange the columns to match the historical datasets
joined_current = joined_current[['date', 'username', 'kcode', 'farmid', 'tree_count', 'primary_key']]

# Preview the new dataset
joined_current.head()

,date,username,kcode,farmid,tree_count,primary_key
0,2019-08-14,demo,klf-2009-0010,crfa345,123,uuid:566c155d-c2b2-4c8e-9da3-f779a55bb6fa
1,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0215,stse143,0,uuid:56f81bd2-b4b3-41fb-ba91-2d26478eeb49
2,2019-08-14,demo,klf-2009-0126,crfa348,56,uuid:66cd257a-09f1-48c8-97b1-b8ac3c5fe308
3,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0162,crfa321,85,uuid:76198455-a30c-4ffd-85ca-2512a53beabe
4,2019-08-14,johnson.ngumbao@komaza.org,klf-2010-0067,crfa335,9,uuid:7ca47cdd-98b0-4d4a-8eea-256c0528c502


In [385]:
# Append current to historical
mer_all_time = joined_historical.append(
    joined_current
)

# Preview the dataset
mer_all_time.head()

,date,username,kcode,farmid,tree_count,primary_key
0,2018-02-01,samson.ngoba@komaza.org,klf-2012-1388,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,2018-02-01,eunice.gambo@komaza.org,klf-2012-1063,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,2018-02-01,eunice.gambo@komaza.org,klf-2011-0676,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,2018-02-01,eunice.gambo@komaza.org,klf-2012-0371,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,2018-02-02,samson.ngoba@komaza.org,klf-2013-0123,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


In [386]:
mer_all_time.tail()

,date,username,kcode,farmid,tree_count,primary_key
304,2019-11-18,johnson.ngumbao@komaza.org,klf-2012-1259,sido36,21.0,uuid:803b60a2-faa2-4569-8118-1530bb3bdaa0
305,2020-01-27,johnson.ngumbao@komaza.org,klf-2010-0316,crfa10,332.0,uuid:319c0df0-0fdf-48e4-bd36-be95e2051508
306,2020-01-28,johnson.ngumbao@komaza.org,klf-2010-0099,erna21,187.0,uuid:19ec04e5-86ca-4c53-8fce-b36a3d44659f
307,2020-01-30,jackson.omolo@komaza.org,klf-2011-0503,emgr8,96.0,uuid:2b2b3809-e7fb-4612-a3ab-41ad1dcf578e
308,2020-02-11,eunice.gambo@komaza.org,klf-2013-0132,duel14,121.0,uuid:eb495e6d-f28a-40b3-8e35-d490c5725327


In [0]:
# Reset index
mer_all_time.reset_index(inplace=True)

In [388]:
mer_all_time.head()

,index,date,username,kcode,farmid,tree_count,primary_key
0,0,2018-02-01,samson.ngoba@komaza.org,klf-2012-1388,erbr62,NaN,uuid:e7955a2a-72c6-40c9-813b-e364761172c6
1,1,2018-02-01,eunice.gambo@komaza.org,klf-2012-1063,erbr64,NaN,uuid:d706a4bb-7d4d-47d0-bce9-70bed43b0c9f
2,2,2018-02-01,eunice.gambo@komaza.org,klf-2011-0676,erbr48,NaN,uuid:3841ddb7-48d5-45d8-b189-d56ea5545e7c
3,3,2018-02-01,eunice.gambo@komaza.org,klf-2012-0371,erbr37,NaN,uuid:546e54ee-3656-4f4c-86b4-b1bd812348cf
4,4,2018-02-02,samson.ngoba@komaza.org,klf-2013-0123,erbr47,NaN,uuid:46f22d59-76e2-4490-b93d-4d368754a8aa


In [0]:
mer_all_time.drop(columns=['index'], inplace=True)

In [390]:
mer_all_time.tail()

,date,username,kcode,farmid,tree_count,primary_key
2725,2019-11-18,johnson.ngumbao@komaza.org,klf-2012-1259,sido36,21.0,uuid:803b60a2-faa2-4569-8118-1530bb3bdaa0
2726,2020-01-27,johnson.ngumbao@komaza.org,klf-2010-0316,crfa10,332.0,uuid:319c0df0-0fdf-48e4-bd36-be95e2051508
2727,2020-01-28,johnson.ngumbao@komaza.org,klf-2010-0099,erna21,187.0,uuid:19ec04e5-86ca-4c53-8fce-b36a3d44659f
2728,2020-01-30,jackson.omolo@komaza.org,klf-2011-0503,emgr8,96.0,uuid:2b2b3809-e7fb-4612-a3ab-41ad1dcf578e
2729,2020-02-11,eunice.gambo@komaza.org,klf-2013-0132,duel14,121.0,uuid:eb495e6d-f28a-40b3-8e35-d490c5725327


In [392]:
# Sort the output by date descending
mer_all_time.sort_values(by=['date'], inplace=True, ascending=False)

# Preview the new dataset
mer_all_time.head()

,date,username,kcode,farmid,tree_count,primary_key
2729,2020-02-11,eunice.gambo@komaza.org,klf-2013-0132,duel14,121.0,uuid:eb495e6d-f28a-40b3-8e35-d490c5725327
2728,2020-01-30,jackson.omolo@komaza.org,klf-2011-0503,emgr8,96.0,uuid:2b2b3809-e7fb-4612-a3ab-41ad1dcf578e
2727,2020-01-28,johnson.ngumbao@komaza.org,klf-2010-0099,erna21,187.0,uuid:19ec04e5-86ca-4c53-8fce-b36a3d44659f
2726,2020-01-27,johnson.ngumbao@komaza.org,klf-2010-0316,crfa10,332.0,uuid:319c0df0-0fdf-48e4-bd36-be95e2051508
2724,2019-11-22,samson.ngoba@komaza.org,klf-2012-0197,erbr72,53.0,uuid:f221554d-2144-4028-8fb2-c8b8280c23c0


### Upload the output to google drive

In [393]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a csv.
mer_all_time.to_csv("mer_all_time.csv", index=False)
uploaded = drive.CreateFile({'title': 'mer_all_time.csv'})
uploaded.SetContentFile("mer_all_time.csv")
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 17dRqA4KfmX3NCaVR97sOKS1dgUVvwZqx
